In [40]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Embedding, Bidirectional, GRU, Dropout, Dense
from tensorflow.keras.models import Model
import matplotlib.pyplot as plt

In [35]:
VOCAB_SIZE = 20000
ENGLISH_SEQUENCE_LENGTH = 64
FRENCH_SEQUENCE_LENGTH = 64
EMBEDDING_DIM = 300
BATCH_SIZE=64

## Data Preparation

In [2]:
!wget https://www.manythings.org/anki/fra-eng.zip

--2023-07-12 11:05:50--  https://www.manythings.org/anki/fra-eng.zip
Resolving www.manythings.org (www.manythings.org)... 173.254.30.110
Connecting to www.manythings.org (www.manythings.org)|173.254.30.110|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7420323 (7.1M) [application/zip]
Saving to: ‘fra-eng.zip’

fra-eng.zip         100%[===================>]   7.08M  4.18MB/s    in 1.7s    

2023-07-12 11:05:53 (4.18 MB/s) - ‘fra-eng.zip’ saved [7420323/7420323]



In [3]:
!unzip "/content/fra-eng.zip" -d "/content/dataset/"

Archive:  /content/fra-eng.zip
  inflating: /content/dataset/_about.txt  
  inflating: /content/dataset/fra.txt  


In [ ]:
!head /content/dataset/fra.txt

Go.	Va !	CC-BY 2.0 (France) Attribution: tatoeba.org #2877272 (CM) & #1158250 (Wittydev)
Go.	Marche.	CC-BY 2.0 (France) Attribution: tatoeba.org #2877272 (CM) & #8090732 (Micsmithel)
Go.	En route !	CC-BY 2.0 (France) Attribution: tatoeba.org #2877272 (CM) & #8267435 (felix63)
Go.	Bouge !	CC-BY 2.0 (France) Attribution: tatoeba.org #2877272 (CM) & #9022935 (Micsmithel)
Hi.	Salut !	CC-BY 2.0 (France) Attribution: tatoeba.org #538123 (CM) & #509819 (Aiji)
Hi.	Salut.	CC-BY 2.0 (France) Attribution: tatoeba.org #538123 (CM) & #4320462 (gillux)
Run!	Cours !	CC-BY 2.0 (France) Attribution: tatoeba.org #906328 (papabear) & #906331 (sacredceltic)
Run!	Courez !	CC-BY 2.0 (France) Attribution: tatoeba.org #906328 (papabear) & #906332 (sacredceltic)
Run!	Prenez vos jambes à vos cous !	CC-BY 2.0 (France) Attribution: tatoeba.org #906328 (papabear) & #2077449 (sacredceltic)
Run!	File !	CC-BY 2.0 (France) Attribution: tatoeba.org #906328 (papabear) & #2077454 (sacredceltic)


## Preprocessing

In [4]:
text_dataset = tf.data.TextLineDataset("/content/dataset/fra.txt")
text_dataset

<TextLineDatasetV2 element_spec=TensorSpec(shape=(), dtype=tf.string, name=None)>

In [ ]:
# find the longest sequence
for i in text_dataset.skip(190000):
    print(len(tf.strings.split(i, " ")))

https://www.tensorflow.org/api_docs/python/tf/keras/layers/TextVectorization?hl=en

Maps text features to integer sequences

In [7]:
english_vectorize_layer = tf.keras.layers.TextVectorization(
    standardize='lower_and_strip_punctuation',
    max_tokens=VOCAB_SIZE,
    output_mode='int',
    output_sequence_length=ENGLISH_SEQUENCE_LENGTH
)

In [8]:
french_vectorize_layer = tf.keras.layers.TextVectorization(
    standardize='lower_and_strip_punctuation',
    max_tokens=VOCAB_SIZE,
    output_mode='int',
    output_sequence_length=FRENCH_SEQUENCE_LENGTH
)

![](https://d2l.ai/_images/seq2seq.svg)

From the file, we will select english(input1), french(input2), french(output)

In [16]:
def selector(input_text):
  split_text = tf.strings.split(input_text, '\t')
  return {'input_1':split_text[0:1], 'input_2': '[start] ' + split_text[1:2]}, split_text[1:2]+' [end]'
  # {english, french(with start)}, french(with end)

In [17]:
split_dataset = text_dataset.map(selector)

In [18]:
for i in split_dataset.take(3):
  print(i)

({'input_1': <tf.Tensor: shape=(1,), dtype=string, numpy=array([b'Go.'], dtype=object)>, 'input_2': <tf.Tensor: shape=(1,), dtype=string, numpy=array([b'[start] Va !'], dtype=object)>}, <tf.Tensor: shape=(1,), dtype=string, numpy=array([b'Va ! [end]'], dtype=object)>)
({'input_1': <tf.Tensor: shape=(1,), dtype=string, numpy=array([b'Go.'], dtype=object)>, 'input_2': <tf.Tensor: shape=(1,), dtype=string, numpy=array([b'[start] Marche.'], dtype=object)>}, <tf.Tensor: shape=(1,), dtype=string, numpy=array([b'Marche. [end]'], dtype=object)>)
({'input_1': <tf.Tensor: shape=(1,), dtype=string, numpy=array([b'Go.'], dtype=object)>, 'input_2': <tf.Tensor: shape=(1,), dtype=string, numpy=array([b'[start] En route !'], dtype=object)>}, <tf.Tensor: shape=(1,), dtype=string, numpy=array([b'En route ! [end]'], dtype=object)>)


Separating to english(x) and french(y) dataset to make a separate vectorizer for each language. This process is to **include [start], [end] tokens in the vectorizer layer**. (With the previous selector function, start/end token is serparate so one of the tokens will be excluded when creating vectorizer)

In [9]:
def separator(input_text):
  split_text = tf.strings.split(input_text, '\t')
  return split_text[0:1], '[start] ' + split_text[1:2]+' [end]'

In [10]:
init_dataset = text_dataset.map(separator)

In [11]:
for i in init_dataset.take(3):
  print(i)

(<tf.Tensor: shape=(1,), dtype=string, numpy=array([b'Go.'], dtype=object)>, <tf.Tensor: shape=(1,), dtype=string, numpy=array([b'[start] Va ! [end]'], dtype=object)>)
(<tf.Tensor: shape=(1,), dtype=string, numpy=array([b'Go.'], dtype=object)>, <tf.Tensor: shape=(1,), dtype=string, numpy=array([b'[start] Marche. [end]'], dtype=object)>)
(<tf.Tensor: shape=(1,), dtype=string, numpy=array([b'Go.'], dtype=object)>, <tf.Tensor: shape=(1,), dtype=string, numpy=array([b'[start] En route ! [end]'], dtype=object)>)


In [12]:
english_training_data = init_dataset.map(lambda x,y : x) # input x,y and output x # only for english
english_vectorize_layer.adapt(english_training_data)

In [15]:
# check the vectorize layer
english_vectorize_layer.get_vocabulary()[832]

'solve'

In [13]:
french_training_data = init_dataset.map(lambda x,y : y) # input x,y and output x # only for english
french_vectorize_layer.adapt(french_training_data)

Coming back to split_dataset created with selector function.

In [19]:
def vectorizer(inputs, output):
  return {'input_1' :english_vectorize_layer(inputs['input_1']),
          'input_2': french_vectorize_layer(inputs['input_2'])}, french_vectorize_layer(output)

In [20]:
dataset = split_dataset.map(vectorizer)

In [22]:
# check if each inputs are mapped with the adapted vectorizer
for i in dataset.take(3):
  print(i)

({'input_1': <tf.Tensor: shape=(1, 64), dtype=int64, numpy=
array([[44,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0]])>, 'input_2': <tf.Tensor: shape=(1, 64), dtype=int64, numpy=
array([[  2, 104,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0]])>}, <tf.Tensor: shape=(1, 64), dtype=int64, numpy=
array([[104,   3,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,

### Batch/Split

In [36]:
dataset=dataset.shuffle(2048).unbatch().batch(BATCH_SIZE).prefetch(buffer_size=tf.data.AUTOTUNE)
NUM_BATCHES=int(200000/BATCH_SIZE)
train_dataset=dataset.take(int(0.9*NUM_BATCHES))
val_dataset=dataset.skip(int(0.9*NUM_BATCHES))

## Modeling

![](https://d2l.ai/_images/seq2seq.svg)

Seq2seq structure

In [23]:
NUM_UNITS = 256

In [32]:
## ENCODER
input = Input(shape=(ENGLISH_SEQUENCE_LENGTH,), dtype="int64", name="input_1") # (BS, 64)
x = Embedding(VOCAB_SIZE, EMBEDDING_DIM, )(input)
encoded_input = Bidirectional(GRU(NUM_UNITS),)(x)

## DECODER
shifted_target = Input(shape=(FRENCH_SEQUENCE_LENGTH,), dtype="int64", name="input_2")
x = Embedding(VOCAB_SIZE, EMBEDDING_DIM,)(shifted_target)

x = GRU(NUM_UNITS*2, return_sequences=True)(x, initial_state=encoded_input)

## OUTPUT
x = Dropout(0.5)(x) # reduce overfitting
target = Dense(VOCAB_SIZE, activation="softmax")(x)
seq2seq_gru = Model([input, shifted_target], target)

#### DECODER
- Why NUM_UNITS*2?
  - Bidirectional layer makes 2 vectors concatenated
- Why return_sequences=True in DECODER GRU?
  - Need the outputs of GRU cell (no need in ENCODER)

#### OUTPUT
- Why VOCAB_SIZE in Dense layer?
  - returned sequence of the GRU cell -> 512 dimension
  - output of GRU layer -> (B, sequence_length, 512)
  - we want to choose one word out of 20000 words (VOCAB_SIZE)
  - So, (B, seq_len, 512) -> (B, seq_len, VOCAB_SIZE)

In [33]:
seq2seq_gru.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 64)]         0           []                               
                                                                                                  
 input_2 (InputLayer)           [(None, 64)]         0           []                               
                                                                                                  
 embedding (Embedding)          (None, 64, 300)      6000000     ['input_1[0][0]']                
                                                                                                  
 embedding_1 (Embedding)        (None, 64, 300)      6000000     ['input_2[0][0]']                
                                                                                              

https://www.tensorflow.org/api_docs/python/tf/keras/losses/SparseCategoricalCrossentropy

- Use this crossentropy loss function when there are **two or more label classes**. We expect labels to be provided as **integers**.
- If you want to provide labels using one-hot representation, please use CategoricalCrossentropy loss.

In [34]:
seq2seq_gru.compile(
    loss=tf.keras.losses.SparseCategoricalCrossentropy(),
    optimizer=tf.keras.optimizers.Adam(1e-4),
    metrics=['accuracy']
)

In [38]:
history = seq2seq_gru.fit(train_dataset, epochs=15, validation_data=val_dataset)

Epoch 1/15
   2337/Unknown - 390s 161ms/step - loss: 0.7498 - accuracy: 0.9168

KeyboardInterrupt: ignored

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

In [ ]:
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()